***INTSALLING NECESSARY DEPENDENCIES***

In [ ]:
!pip -q install streamlit==1.37.0 streamlit-drawable-canvas==0.9.3 pyngrok==7.2.3 opencv-python-headless==4.10.0.84


***LOADING SAVED MODEL***

In [ ]:
from google.colab import files
uploaded = files.upload()

import shutil
if "mnist_dense.keras" not in uploaded:
    # rename if uploaded file has different name
    src = list(uploaded.keys())[0]
    shutil.move(src, "mnist_dense.keras")
print("✅ Model saved as mnist_dense.keras")


***CREATING STREAMLIT APP***

In [ ]:
%%writefile app.py
import os
import cv2
import numpy as np
import streamlit as st
from streamlit_drawable_canvas import st_canvas
import tensorflow as tf

st.set_page_config(page_title="Digit Recognition", page_icon="🖌️", layout="centered")

@st.cache_resource
def load_model():
    return tf.keras.models.load_model("mnist_dense.keras", compile=False)

model = load_model()

def preprocess(img_rgba):
    # RGBA → Gray
    gray = cv2.cvtColor(img_rgba.astype(np.uint8), cv2.COLOR_RGBA2GRAY)
    inv = 255 - gray  # invert → white digit on black background

    # Crop bounding box
    ys, xs = np.where(inv > 0)
    if len(xs) == 0 or len(ys) == 0:
        return np.zeros((28,28), np.float32)
    x1, x2, y1, y2 = xs.min(), xs.max(), ys.min(), ys.max()
    crop = inv[y1:y2+1, x1:x2+1]

    # Resize to 20x20, then pad to 28x28
    h, w = crop.shape
    if h > w:
        new_h, new_w = 20, int(w*(20/h))
    else:
        new_w, new_h = 20, int(h*(20/w))
    resized = cv2.resize(crop, (new_w, new_h))
    top, bottom = (28-new_h)//2, 28-new_h-(28-new_h)//2
    left, right = (28-new_w)//2, 28-new_w-(28-new_w)//2
    canvas = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
    return canvas.astype(np.float32)/255.0

def prepare_input(img28):
    shape = model.input_shape
    if len(shape)==2: # Dense (784,)
        return img28.reshape(1,784)
    else: # CNN (28,28,1)
        return img28.reshape(1,28,28,1)

st.title("🖌️ Digit Drawing Recognition")
st.write("Draw a digit below and get prediction:")

if "canvas_key" not in st.session_state:
    st.session_state.canvas_key = 0

canvas_result = st_canvas(
    stroke_color="#000000",
    stroke_width=15,
    background_color="#FFFFFF",
    height=280,
    width=280,
    drawing_mode="freedraw",
    key=f"canvas{st.session_state.canvas_key}"
)

if canvas_result.image_data is not None:
    img28 = preprocess(canvas_result.image_data)
    st.image((img28*255).astype(np.uint8), width=150, caption="28x28 Input")

    if st.button("Predict"):
        x = prepare_input(img28)
        preds = model.predict(x, verbose=0)[0]
        st.success(f"Prediction: {np.argmax(preds)} (Confidence {np.max(preds):.2%})")
        st.bar_chart({str(i): float(preds[i]) for i in range(10)})

if st.button("Clear / Try Again"):
    st.session_state.canvas_key += 1
    st.rerun()


***USING NGROK FOR STREAMLIT CANVAS***

In [ ]:
NGROK_AUTH = "ENTER YOUR AUTHROKEN HERE"


***GENERATING LINK FOR PREDICTION***

In [ ]:
from pyngrok import ngrok
import subprocess, time, os, signal

# Kill previous if any
!pkill streamlit || echo "no streamlit running"

# Set auth
if NGROK_AUTH and "PASTE_YOUR_TOKEN_HERE" not in NGROK_AUTH:
    ngrok.set_auth_token(NGROK_AUTH)

# Start streamlit
proc = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "0.0.0.0"])

# Wait and start tunnel
time.sleep(3)
public_url = ngrok.connect(8501).public_url
print("🌍 Your app is live at:", public_url)
